In [30]:
cd ..

/Users/bastianlengen/Desktop/NameToBeFound


In [32]:
import pandas as pd
import numpy as np
import Fit_parameters as Fp

def load_data(work_dir='./'):
    '''
    Loads and return a dictionary of pandas DataFrame of the different .csv

    :type   work_dir: string
    :param  work_dir:
    '''
    Data_dir = work_dir + 'Data/'

    # Create an empty dictionnary of dataset
    DF_dict={}

    # Loads the Cepheids related DataFrame
    if Fp.include_Cepheids==True:
        print('Loading Cepheids & Cepheids anchors...')
        DF_dict['Cepheids'] = pd.read_csv(Data_dir+'Cepheids.csv', sep=',')
        DF_dict['Cepheids'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z', 'V-I']
        DF_dict['Cepheids_anchors'] = pd.read_csv(Data_dir + 'Cepheids_anchors.csv', sep=',')
        DF_dict['Cepheids_anchors'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z', 'V-I', 'mu', 'sig_mu']
        if Fp.include_MW==True:
            print('Loading MW Cepheids...')
            DF_dict['Cepheids_MW'] = pd.read_csv(Data_dir+'Cepheids_MW.csv', sep=',')
            DF_dict['Cepheids_MW'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z', 'V-I', 'pi', 'sig_pi']
        print('Loading SNe for the Cepheids-host galaxies...')
        DF_dict['SNe_Cepheids'] = pd.read_csv(Data_dir + 'SNe_Cepheids.csv', sep=',')
        DF_dict['SNe_Cepheids'].columns = ['Gal', 'mB', 'sig_mB']

    # Loads the TRGB related DataFrame
    if Fp.include_TRGB==True:
        print('Loading TRGB & TRGB anchors...')
        DF_dict['TRGB'] = pd.read_csv(Data_dir+'TRGB.csv', sep=',')
        DF_dict['TRGB'].columns = ['Gal', 'm', 'sig_m', 'A', 'V-I', 'z']
        DF_dict['TRGB_anchors'] = pd.read_csv(Data_dir + 'TRGB_anchors.csv', sep=',')
        DF_dict['TRGB_anchors'].columns = ['Gal', 'm', 'sig_m', 'A', 'V-I', 'z', 'mu', 'sig_mu']
        print('Loading SNe for the TRGB-host galaxies...')
        DF_dict['SNe_TRGB'] = pd.read_csv(Data_dir + 'SNe_TRGB.csv', sep=',')
        DF_dict['SNe_TRGB'].columns = ['Gal', 'mB', 'sig_mB']

    # Loads the SNe for the aB fit
    if Fp.fit_aB==True:
        print('Loading SNe for the aB fit...')
        DF_dict['SNe_Hubble'] = pd.read_csv(Data_dir + 'SNe_Hubble.csv', sep=',')
        DF_dict['SNe_Hubble'].columns = ['name', 'mB', 'sig_mB', 'z', 'sig_z']

    return DF_dict

DF_dict = load_data()

Loading Cepheids & Cepheids anchors...
Loading MW Cepheids...
Loading SNe for the Cepheids-host galaxies...
Loading SNe for the aB fit...


In [3]:
def fit_distance_ladder(DF_dict, work_dir='./'):
    '''
    Return the raw result of the fit parameters

    :type   DF_dict: dictionary of pandas DataFrame
    :param  DF_dict: Dictionary that contains the DataFrame that will be fitted.
    :type   work_dir: string
    :param  work_dir: Working directory that contains the Data/ folder.
    '''
    Data_dir = work_dir + 'Data/'

    ### Useful functions
    # function that change a horizontal vector to a vertical vector
    def vert(v):
        v = np.array(v)
        return v.reshape(len(v),1)
    def logczexpansion(z):
        return np.log10(Fp.c*z*(1 + 1/2*(1-Fp.q0)*z - 1/6*(1-Fp.q0-3*Fp.q0**2+Fp.j0)*z**2))
    
    
    ### We first create an empty signal vector y, design matrix L and parameters vector q and diag (for Sigma)
    [y, L, diag] = 3*[np.empty(0)]
    q_string = []
    
    # For the Cepheids
    if Fp.include_Cepheids == True:
        ### Load the DF
        # Use the Cepheids DF and check for its consistency with the Fit_parameters.py file
        Cepheids = DF_dict['Cepheids']
        galaxies_Cep = Cepheids['Gal'].drop_duplicates().reset_index(drop=True) # List of Cepheids-host galaxy
        if len(galaxies_Cep)!=Fp.N_galaxies_Cep:
            print('ERROR: make sure that the value for N_galaxies_Cep in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the Cepheids.csv file!')
            return
        # The anchors Cepheids
        Cepheids_anchors = DF_dict['Cepheids_anchors']
        anchors_Cep =  Cepheids_anchors['Gal'].drop_duplicates().reset_index(drop=True) # List of Cepheids-host galaxy
        if len(anchors_Cep)!=Fp.N_anchors_Cep:
            print('ERROR: make sure that the value for N_anchors_Cep in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the Cepheids_anchors.csv file!')
            return
        # The MW Cepheids
        if Fp.include_MW == True:
            Cepheids_MW = DF_dict['Cepheids_MW']  
        # The SNe from the Cepheid-host galaxy
        SNe_Cepheids = DF_dict['SNe_Cepheids']

        ### Complete y & diag for the covariance matrix
        y = np.append(y, Cepheids['mW']) # Cepheids 
        err = (Cepheids['sig_mW']+Fp.added_scatter)**2
        diag = np.append(diag, err)
        y = np.append(y, Cepheids_anchors['mW']-Cepheids_anchors['mu']) # Cepheids anchors
        err = (Cepheids_anchors['sig_mW']+Fp.added_scatter)**2+Cepheids_anchors['sig_mu']**2
        diag = np.append(diag, err)
        y = np.append(y, np.zeros([1,Fp.N_anchors_Cep])) # External constrainte (Delta mu_anchors = 0)
        index = Cepheids_anchors['Gal'].drop_duplicates().index # find index of the 1st appearance of the anchor galaxy in the DF
        err =  Cepheids_anchors.loc[index, 'sig_mu'].values**2
        diag= np.append(diag,err)
        # Milky Way Cepheids
        if Fp.include_MW == True:
            if Fp.fixed_zp == True:
                y = np.append(y, Cepheids_MW['mW'] \
                              - 10 + 5 * np.log10(Cepheids_MW['pi'])\
                              + 5/np.log(10)*Fp.zp/Cepheids_MW['pi'])
                err = (Cepheids_MW['sig_mW']+Fp.added_scatter)**2 \
                    + ((5/np.log(10)/Cepheids_MW['pi'] - \
                       5/np.log(10)*Fp.zp/Cepheids_MW['pi']**2)*Cepheids_MW['sig_pi'])**2 \
                    + (5/np.log(10)/Cepheids_MW['pi']*Fp.sig_zp)**2
                diag = np.append(diag, err)
            else:
                y = np.append(y, Cepheids_MW['mW'] \
                              - 10 + 5 * np.log10(Cepheids_MW['pi']))
                err = (Cepheids_MW['sig_mW']+Fp.added_scatter)**2 \
                    + ((5/np.log(10)/Cepheids_MW['pi']*Cepheids_MW['sig_pi']))**2
                diag = np.append(diag, err)
        if Fp.fixed_Zw == True:
            minus_MH = Cepheids['M/H']
            minus_MH = np.append(minus_MH, Cepheids_anchors['M/H'])
            minus_MH = np.append(minus_MH, np.zeros([1,Fp.N_anchors_Cep]))
            if Fp.include_MW == True:
                minus_MH = np.append(minus_MH, Cepheids_MW['M/H'])
            y = y-Fp.Zw*minus_MH
            diag = diag + (Fp.sig_Zw*minus_MH)**2
        y = np.append(y, SNe_Cepheids['mB'])
        err = SNe_Cepheids['sig_mB']**2
        diag = np.append(diag, err)
            

        ### Complete q_string for future dict so it's easier to access
        # The parameters will be in that ordre:
        # mu, MW, b, (bs), (zw), (zp)
        for gal in galaxies_Cep:
            q_string.append(f'mu_{gal}')
        for gal in anchors_Cep:
            q_string.append(f'Dmu_{gal}')
        q_string.append('MW')
        q_string.append('MB')
        if Fp.PLR_break == True:
            q_string.append('b_s')
            q_string.append('b_l')
        else:
            q_string.append('b')
        if Fp.fixed_Zw == False:
            q_string.append('Zw')
        if ((Fp.include_MW == True) and (Fp.fixed_zp==False)):
            q_string.append('zp')


        ### fill L
        # First create the matrix with just mu, Dmu, M, MB and b 
        L = np.zeros([len(y), len(q_string)])
        # Iterate over each line, start with Cepheids
        index_offset = 0
        for i in range(len(Cepheids)):
            # mu
            gal = Cepheids.loc[i,'Gal']
            index = q_string.index(f'mu_{gal}')
            L[i+index_offset,index] = 1
            # MW
            index = q_string.index('MW')
            L[i+index_offset,index] = 1
            # logP
            if Fp.PLR_break == False:
                index = q_string.index('b')
            else:
                threshold = np.log10(Fp.break_P)
                if Cepheids.loc[i, 'logP'] < threshold:
                    index = q_string.index('b_s')
                else:
                    index = q_string.index('b_l')
            L[i+index_offset,index] = Cepheids.loc[i,'logP']
            # Zw
            if Fp.fixed_Zw == False:
                index = q_string.index('Zw')
                L[i+index_offset,index] = Cepheids.loc[i,'M/H']
        # Anchors Cepheids
        index_offset += i+1
        for i in range(len(Cepheids_anchors)):
            # Dmu
            gal = Cepheids_anchors.loc[i,'Gal']
            index = q_string.index(f'Dmu_{gal}')
            L[i+index_offset,index] = 1
            # MW
            index = q_string.index('MW')
            L[i+index_offset,index] = 1
            # logP
            if Fp.PLR_break == False:
                index = q_string.index('b')
            else:
                threshold = np.log10(Fp.break_P)
                if Cepheids_anchors.loc[i, 'logP'] < threshold:
                    index = q_string.index('b_s')
                else:
                    index = q_string.index('b_l')
            L[i+index_offset,index] = Cepheids_anchors.loc[i,'logP']
            # Zw
            if Fp.fixed_Zw == False:
                index = q_string.index('Zw')
                L[i+index_offset,index] = Cepheids_anchors.loc[i,'M/H']
        # External constriant (Delta mu)
        index_offset += i+1
        for i in range(len(anchors_Cep)):
            gal = anchors_Cep[i]
            # Dmu
            index = q_string.index(f'Dmu_{gal}')
            L[i+index_offset,index] = 1
        # MW Cepheids
        if Fp.include_MW == True:
            index_offset += i+1
            for i in range(len(Cepheids_MW)):
                # MW
                index = q_string.index('MW')
                L[i+index_offset,index] = 1
                # logP
                if Fp.PLR_break == False:
                    index = q_string.index('b')
                else:
                    threshold = np.log10(Fp.break_P)
                    if Cepheids_MW.loc[i, 'logP'] < threshold:
                        index = q_string.index('b_s')
                    else:
                        index = q_string.index('b_l')
                L[i+index_offset,index] = Cepheids_MW.loc[i,'logP']
                # Zw
                if Fp.fixed_Zw == False:
                    index = q_string.index('Zw')
                    L[i+index_offset,index] = Cepheids_MW.loc[i,'M/H']
                # zp
                if Fp.fixed_zp == False:
                    index = q_string.index('zp')
                    L[i+index_offset,index] = - 5/np.log(10)/Cepheids_MW.loc[i,'pi']
        # SNe from Cepheids-host galaxies
        index_offset += i+1
        for i in range(len(SNe_Cepheids)):
            # mu
            gal = SNe_Cepheids.loc[i,'Gal']
            index = q_string.index(f'mu_{gal}')
            L[i+index_offset,index] = 1
            # MB
            index = q_string.index('MB')
            L[i+index_offset,index] = 1
    
    # For the TRGB
    if Fp.include_TRGB == True:
        ### Load the DF
        # TRGB
        TRGB = DF_dict['TRGB']
        galaxies_TRGB = TRGB['Gal'].drop_duplicates().reset_index(drop=True) # List of TRGB-host galaxies
        if len(galaxies_TRGB)!=Fp.N_galaxies_TRGB:
            print('ERROR: make sure that the value for N_galaxies_TRGB in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the TRGB.csv file!')
        # TRGB anchors
        TRGB_anchors = DF_dict['TRGB_anchors']
        anchors_TRGB = TRGB_anchors['Gal'].drop_duplicates().reset_index(drop=True) # List of TRGB-host galaxies
        if len(anchors_TRGB)!=Fp.N_anchors_TRGB:
            print('ERROR: make sure that the value for N_anchors_TRGB in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the TRGB_anchors.csv file!')
        # SNe_TRGB
        SNe_TRGB = DF_dict['SNe_TRGB']
            
        ### Complete y
        # TRGB
        add_y = np.array(TRGB['m']-TRGB['A'])
        err = TRGB['sig_m']**2+(0.5*TRGB['A'])**2
        diag = np.append(diag,err)
        # TRGB anchors
        add_y = np.append(add_y,TRGB_anchors['m']-TRGB_anchors['A']-TRGB_anchors['mu'])
        err = TRGB_anchors['sig_m']**2+(0.5*TRGB_anchors['A'])**2+TRGB_anchors['sig_mu']**2
        diag = np.append(diag,err)
        if Fp.use_color == True:
            color = TRGB['V-I']-Fp.mid_VI
            color = np.append(color, TRGB_anchors['V-I']-Fp.mid_VI)
            add_y = add_y-0.2*color
        y = np.append(y,add_y)
        # External constraint (Delta mu)
        y = np.append(y, np.zeros([1,Fp.N_anchors_TRGB])) # External constrainte (Delta mu_anchors = 0)
        index = TRGB_anchors['Gal'].drop_duplicates().index # find index of the 1st appearance of the anchor galaxy in the DF
        err =  TRGB_anchors.loc[index, 'sig_mu'].values**2
        diag = np.append(diag,err)
        # SNe TRGB
        y = np.append(y,SNe_TRGB['mB'])
        err = SNe_TRGB['sig_mB']**2
        diag = np.append(diag,err)
        
        ### Complete q
        # (Delta mu), mu, MTRGB
        if Fp.include_Cepheids == False:
            for gal in galaxies_TRGB:
                q_string.append(f'mu_{gal}')
            for gal in anchors_TRGB:
                q_string.append(f'Dmu_{gal}')
            q_string.append('MTRGB')
            q_string.append('MB') # Have to declare it here if there is no Cepheid
        else:
            for gal in galaxies_TRGB:
                if gal in list(galaxies_Cep):
                    if Fp.different_mu == True:
                        q_string.append(f'Dmu_{gal}') # Delta mu if in both sets of galaxies
                else:
                    q_string.append(f'mu_{gal}')  
            for gal in anchors_TRGB:
                if gal not in list(anchors_TRGB):
                    q_string.append(f'Dmu_{gal}')
            q_string.append('MTRGB')
            if Fp.different_MB == True:
                q_string.append('DMB')
            
        ### Complete L
        # TRGB
        to_add = np.zeros([len(TRGB)+len(TRGB_anchors)+Fp.N_anchors_TRGB+len(SNe_TRGB),len(q_string)])
        if Fp.include_Cepheids == False:
            index_offset = 0
            # TRGB
            for i in range(len(TRGB)):
                # mu
                gal = TRGB.loc[i,'Gal']
                index = q_string.index(f'mu_{gal}')
                to_add[i+index_offset,index] = 1
                # MTRGB
                index = q_string.index('MTRGB')
                to_add[i+index_offset,index] = 1
            index_offset += i+1
            # TRGB anchors
            for i in range(len(TRGB_anchors)):
                # Dmu
                gal = TRGB_anchors.loc[i,'Gal']
                index = q_string.index(f'Dmu_{gal}')
                to_add[i+index_offset,index] = 1
                # MTRGB
                index = q_string.index('MTRGB')
                to_add[i+index_offset,index] = 1
            index_offset += i+1
            # External constriant (Delta mu)
            for i in range(len(TRGB_anchors)):
                # Dmu
                gal = TRGB_anchors.loc[i,'Gal']
                index = q_string.index(f'Dmu_{gal}')
                to_add[i+index_offset,index] = 1
            index_offset += i+1
            # SNe from TRGB-host galaxies
            for i in range(len(SNe_TRGB)):
                # mu
                index = galaxies_TRGB[galaxies_TRGB==SNe_TRGB.loc[i,'Gal']].index[0]
                to_add[i+index_offset,index] = 1
                # MB
                index = q_string.index('MB')
                to_add[i+index_offset,index] = 1
            L = to_add

        else:
            index_offset = 0
            # TRGB
            for i in range(len(TRGB)):
                # mu & Dmu
                gal = TRGB.loc[i,'Gal']
                if gal in list(galaxies_TRGB) and gal in list(galaxies_Cep):
                    index = q_string.index(f'mu_{gal}')
                    to_add[i+index_offset,index] = 1
                    if Fp.different_mu == True:
                        index = q_string.index(f'Dmu_{gal}')
                else:
                    index = q_string.index(f'mu_{gal}')
                    to_add[i+index_offset,index] = 1 
                # MTRGB
                index = q_string.index('MTRGB')
                to_add[i+index_offset,index] = 1
            index_offset += i+1
            # TRGB anchors
            for i in range(len(TRGB_anchors)):
                # Dmu
                gal = TRGB_anchors.loc[i,'Gal']
                index = q_string.index(f'Dmu_{gal}')
                to_add[i+index_offset,index] = 1
                # MTRGB
                index = q_string.index('MTRGB')
                to_add[i+index_offset,index] = 1
            index_offset += i+1
            # External constriant (Delta mu)
            for i in range(len(anchors_TRGB)):
                gal = anchors_TRGB[i]
                # Dmu
                index = q_string.index(f'Dmu_{gal}')
                to_add[i+index_offset,index] = 1
            # SNe from TRGB-host galaxies
            index_offset += i+1
            for i in range(len(SNe_TRGB)):
                # mu & Dmu
                gal = SNe_TRGB.loc[i,'Gal']
                if gal in list(galaxies_TRGB) and gal in list(galaxies_Cep):
                    index = q_string.index(f'mu_{gal}')
                    to_add[i+index_offset,index] = 1
                    if Fp.different_mu == True:
                        index = q_string.index(f'Dmu_{gal}')
                else:
                    index = q_string.index(f'mu_{gal}')
                    to_add[i+index_offset,index] = 1 
                # MB
                index = q_string.index('MB')
                to_add[i+index_offset,index] = 1
                
            # Add a few lines to the right of the L matrix for the added parameters
            missing_columns = to_add.shape[1]-L.shape[1]
            L = np.hstack([L,np.zeros([len(L),missing_columns])])
            L = np.vstack([L,to_add])
        

            
    # For the aB fit
    if Fp.fit_aB == True:
        ### Load the DF 
        SNe_Hubble = DF_dict['SNe_Hubble'] 
        SNe_Hubble = SNe_Hubble[(SNe_Hubble['z']>Fp.z_min)&(SNe_Hubble['z']<Fp.z_max)].reset_index(drop=True)
        
        ### Complete y
        y = np.append(y,SNe_Hubble['mB']-5*logczexpansion(SNe_Hubble['z'])-25)
        err = SNe_Hubble['mB']**2
        diag = np.append(diag,err)
        
        ### Complete q
        q_string.append('5logH0')
        
        ### Complete L 
        # First add a column for the 1 extra parameters to the already existing matrice L
        L = np.hstack([L,np.zeros([len(L),1])])
        # Then add the lines
        to_add = np.zeros([len(SNe_Hubble), len(q_string)])
        # MB columns
        to_add[:,q_string.index('MB')] = 1
        # logH0 columns
        to_add[:,q_string.index('5logH0')] = -1
        L = np.vstack([L,to_add])



    ### Solve
    C = np.diag(diag)

    ### Find the optimal parameters :
    LT = np.transpose(L)
    C1 = np.linalg.inv(C)
    cov = np.linalg.inv(np.matmul(np.matmul(LT, C1), L))
    q = np.matmul(np.matmul(np.matmul(cov, LT), C1), y)
    chi2 = np.matmul(np.matmul(np.transpose(y - np.matmul(L, q)), C1), y - np.matmul(L, q))
    dof = len(y)-len(q)
    chi2_reduced = chi2/dof
    
    q_dict = dict(zip(q_string,q))
    return y, q_dict, L, chi2_reduced

y, q_dict, L, chi2_reduced = fit_distance_ladder(DF_dict)

In [6]:
pd.read_csv('Data/H1PStars/Cepheids_MW.csv')

,Name,Period,mh,mh_error,Parallax,Parallax_error,FE/H,FE/H_Eerr
0,CM_Sct,3.917025,7.239755,0.062437,0.442947,0.010148,NaN,NaN
1,CG_Cas,4.365540,7.774535,0.062437,0.335703,0.010634,0.09,0.06
2,CF_Cas,4.875220,7.608335,0.056883,0.322512,0.010674,0.02,0.06
3,VW_Cru,5.265220,5.681213,0.062437,0.731787,0.011574,0.19,0.06
4,CV_Mon,5.378670,6.165182,0.056883,0.585257,0.012235,0.09,0.09
5,V_CEN,5.493930,4.249141,0.056883,1.336575,0.009765,0.04,0.09
6,CS_Vel,5.904740,7.642653,0.056883,0.281177,0.010598,0.12,0.06
7,X_Cru,6.220090,5.717240,0.062437,0.639288,0.010113,0.15,0.06
8,X_Vul,6.319580,4.832108,0.056906,0.879557,0.010260,0.07,0.08
9,U_Sgr,6.745226,3.636512,0.056883,1.554007,0.009834,0.08,0.08
